In [1]:
import Cython
%load_ext Cython

In [2]:
import time
import numpy as np
from tqdm import tqdm_notebook as tqdm

# Базовый пример использования Cython
В данном пункте приведем пример простой функции на языке python. В качестве функции рассматривалась функция, которая суммирует числа от $0$ до $N$. В эксперименте сравнивается скорость работы данной функции в следующих случаях: 
* функция реализована на python без предварительной компиляции; 
* реализована на python но предварительно скомпилирована;
* реализована на cython с типизацией объектов.

## Модели

In [3]:
%%cython

def CythonFunc(N):
    ret = 0
    for n in range(N):
        ret += n
    return ret

def CythonTypedFunc(int N):
    cdef long ret = 0
    cdef long n
    for n in range(N):
        ret += n
    return ret

In [4]:
def func(N):
    ret = 0
    for n in range(N):
        ret += n
    return ret

## Comparison
Для сравнения рассматривается $N = 10000000$ для всех моделей, а также каждая функция вызывается $200$ раз для усреднения результата.

In [5]:
N = 10000000
number_of_experiment = 200

#### Python Func
Время работы функции для базовой реализации на python.

In [6]:
list_of_times = []
############################################

iterable = tqdm(range(number_of_experiment))
for _ in iterable:
    start = time.time()
    func(N)
    end = time.time()
    
    list_of_times.append(end-start)
    
    iterable.set_postfix_str('(time={})'.format(np.mean(list_of_times)))

#### Cython Func
Время работы скомпилированной программы на cython без добавления типизации объектов.

In [7]:
list_of_times = []
############################################

iterable = tqdm(range(number_of_experiment))
for _ in iterable:
    start = time.time()
    CythonFunc(N)
    end = time.time()
    
    list_of_times.append(end-start)
    
    iterable.set_postfix_str('(time={})'.format(np.mean(list_of_times)))

#### Cython Typed Func
Время работы скомпилированной программы на cython c добавлениям типизации объектов.

In [8]:
list_of_times = []
############################################

iterable = tqdm(range(number_of_experiment))
for _ in iterable:
    start = time.time()
    ret = CythonTypedFunc(N)
    end = time.time()
    
    list_of_times.append(end-start)
    
    iterable.set_postfix_str('(time={})'.format(np.mean(list_of_times)))

В данном тривиальном примере получаем следующее:

| Функция  | Время |
| ------------- | ------------- |
| Python Func  | 583 ms  |
| Cython Func  | 435 ms  |
| Cython Typed Func  | 0.151 ms |


# NumPy in Cython

In [9]:
%%cython
import numpy as np
cimport numpy as np
np.import_array()

cpdef return_zeros():
    return np.zeros([10, 15], dtype=np.int32)

cpdef return_empty():
    cdef np.npy_intp dims[2]
    dims[0] = 10
    dims[1] = 15
    cdef np.ndarray data = np.PyArray_SimpleNew(2, dims, np.NPY_INT32)
    np.PyArray_FILLWBYTE(data, 0)
    return data

# WordEmbeding [CountVectorizer]

## Data

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
import re
import numpy as np

In [12]:
with open('./data/ru.tok', 'r') as f:
    corpus = f.read().lower().splitlines()

## Models

### Python Vectorizer

In [13]:
token_pattern = re.compile(r"(?u)\b\w\w+\b")
class Vectorizer:
    def __init__(self):
        self.WordToInt = dict()
        pass
    
    def split_sent(self, sent):
        return token_pattern.findall(sent)
    
    def _fit(self, corpus):
        for sent in corpus:
            words = self.split_sent(sent)
            for word in words:
                if word not in self.WordToInt:
                    self.WordToInt[word] = len(self.WordToInt)
        return
    
    def fit(self, corpus):
        return self._fit(corpus)
    
    def get_feature_names(self):
        return list(self.WordToInt.keys())
    
    def _transform(self, corpus):
        transformed = np.zeros([len(corpus), len(self.WordToInt)], dtype=np.int64)
        for i, sent in enumerate(corpus):
            words = self.split_sent(sent)
            row = transformed[i]
            LocalVocab = dict()
            for word in words:
                if word in self.WordToInt:
                    ind = self.WordToInt[word]
                    if ind not in LocalVocab:
                        LocalVocab[ind] = 1
                    else:
                        LocalVocab[ind] += 1
                    
            for ind in LocalVocab:
                row[ind]=LocalVocab[ind];
                    
        return transformed
    
    def transform(self, corpus):
        return self._transform(corpus)
    
    def fit_transform(self, corpus):
        self._fit(corpus)
        return self._transform(corpus)

### Cython Vectorizer and Cython+Typed Vectorizer

In [14]:
%%cython
import re
import numpy as np
cimport numpy as np
np.import_array()

token_pattern = re.compile(r"(?u)\b\w\w+\b")

class CythonVectorizer:
    def __init__(self):
        self.WordToInt = dict()
        pass
    
    def split_sent(self, sent):
        return token_pattern.findall(sent)
    
    def fit(self, corpus):
        for sent in corpus:
            words = self.split_sent(sent)
            for word in words:
                if word not in self.WordToInt:
                    self.WordToInt[word] = len(self.WordToInt)
        return
    
    def get_feature_names(self):
        return list(self.WordToInt.keys())
    
    def transform(self, corpus):
        transformed = np.zeros([len(corpus), len(self.WordToInt)], dtype=np.int64)
        for i, sent in enumerate(corpus):
            words = self.split_sent(sent)
            row = transformed[i]
            LocalVocab = dict()
            for word in words:
                if word in self.WordToInt:
                    ind = self.WordToInt[word]
                    if ind not in LocalVocab:
                        LocalVocab[ind] = 1
                    else:
                        LocalVocab[ind] += 1
                    
            for ind in LocalVocab:
                row[ind]=LocalVocab[ind];
                    
        return transformed
    
    def fit_transform(self, corpus):
        self._fit(corpus)
        return self._transform(corpus)




cdef class CythonTypedVectorizer:
    cdef dict WordToInt
    def __cinit__(self):
        self.WordToInt = dict()
        pass
        
    cdef list split_sent(self, str sent):
        return token_pattern.findall(sent)
    
    cpdef void fit(self, list corpus):
        cdef str word, sent
        cdef list words
        for sent in corpus:
            words = self.split_sent(sent)
            for word in words:
                if word not in self.WordToInt:
                    self.WordToInt[word] = len(self.WordToInt)
        return
    
    def get_feature_names(self):
        return list(self.WordToInt.keys())
    
    cpdef np.ndarray transform(self, list corpus):
        cdef np.ndarray row
        cdef dict LocalVocab
        cdef str word, sent
        cdef int i, ind
        cdef list words
        cdef np.ndarray transformed = np.zeros([len(corpus), len(self.WordToInt)], dtype=np.int64)
        for i, sent in enumerate(corpus):
            words = self.split_sent(sent)
            row = transformed[i]
            LocalVocab = dict()
            for word in words:
                if word in self.WordToInt:
                    ind = self.WordToInt[word]
                    if ind not in LocalVocab:
                        LocalVocab[ind] = 1
                    else:
                        LocalVocab[ind] += 1
                    
            for ind in LocalVocab:
                row[ind]=LocalVocab[ind];
                    
        return transformed
    
    def fit_transform(self, list corpus):
        self._fit(corpus)
        return self._transform(corpus)

## Comparison 

In [15]:
number_of_experiment = 200

#### Sklearn

In [16]:
list_of_fit_times = []
list_of_transform_times = []
############################################

iterable = tqdm(range(number_of_experiment))
for _ in iterable:
    model = CountVectorizer()
    
    start = time.time()
    model.fit(corpus)
    end = time.time()
    
    list_of_fit_times.append(end-start)
    
    start = time.time()
    _ = model.transform(corpus).toarray()
    end = time.time()
    
    list_of_transform_times.append(end-start)
    
    iterable.set_postfix_str('(fit={}, transform={})'.format(np.mean(list_of_fit_times), np.mean(list_of_transform_times)))

#### Python Vectorizer

In [17]:
list_of_fit_times = []
list_of_transform_times = []
############################################

iterable = tqdm(range(number_of_experiment))
for _ in iterable:
    model = Vectorizer()
    
    start = time.time()
    model.fit(corpus)
    end = time.time()
    
    list_of_fit_times.append(end-start)
    
    start = time.time()
    _ = model.transform(corpus)
    
    end = time.time()
    
    list_of_transform_times.append(end-start)
    
    iterable.set_postfix_str('(fit={}, transform={})'.format(np.mean(list_of_fit_times), np.mean(list_of_transform_times)))

#### Cython Vectorizer without Typing

In [18]:
list_of_fit_times = []
list_of_transform_times = []
############################################

iterable = tqdm(range(number_of_experiment))
for _ in iterable:
    model = CythonVectorizer()
    
    start = time.time()
    model.fit(corpus)
    end = time.time()
    
    list_of_fit_times.append(end-start)
    
    start = time.time()
    _ = model.transform(corpus)
    end = time.time()
    
    list_of_transform_times.append(end-start)
    
    iterable.set_postfix_str('(fit={}, transform={})'.format(np.mean(list_of_fit_times), np.mean(list_of_transform_times)))

#### Cython Vectorizer with Typing

In [19]:
list_of_fit_times = []
list_of_transform_times = []
############################################

iterable = tqdm(range(number_of_experiment))
for _ in iterable:
    model = CythonTypedVectorizer()
    
    start = time.time()
    model.fit(corpus)
    end = time.time()
    
    list_of_fit_times.append(end-start)
    
    start = time.time()
    _ = model.transform(corpus)
    end = time.time()
    
    list_of_transform_times.append(end-start)
    
    iterable.set_postfix_str('(fit={}, transform={})'.format(np.mean(list_of_fit_times), np.mean(list_of_transform_times)))

Оценка времени выполнялась для выборки, которая состояла из $79582$ строк ($16.8$mb текста). Всего в данном тексте содержится $76777$ различных токенов, которые были найдены всемы моделями и добавлены в словарь.


Для чистоты эксперимента время предаставленное в таблице является устредненным по $200$ вызовам функции ```fit``` и $200$ вызовам функции ```transform```.

| Функция  | Время fit | Время transform |
| ------------- | ------------- | ------------- |
| CountVectorizer  | 1401 ms  | 2521 ms |
| Python Vectorizer  | 623 ms | 2152 ms |
| Cython Vectorizer  | 570 ms | 1935 ms |
| CythonTyped Vectorizer  | 532 ms | 1868 ms |

